In [1]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# check if USE_OLLAMA is True
if os.getenv('USE_OLLAMA') == 'True':
    print('Using OLLAMA')
if os.getenv('USE_OPENAI') == 'True':
    print('Using OpenAI') 




Using OLLAMA


In [2]:
import os
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
#model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")
model = ChatOllama(model="llama2")

output_parser = StrOutputParser()


Use document loaders to load data from a source as Document

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [19]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


you can also load directory using directory loader.
and split a long document into smaller chunks that can fit into your model's context window.

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


raw_documents = DirectoryLoader('../article', glob="*.pdf", loader_cls=PyPDFLoader).load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)



Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [5]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings


#vectorstore = FAISS.from_documents(documents, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))
vectorstore = FAISS.from_documents(documents, embedding=GPT4AllEmbeddings())

retriever = vectorstore.as_retriever()


invoke question

In [6]:
from pprint import pprint

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | model 
    | output_parser
)

pprint(chain.invoke("tell me about NVMe"))

("Sure, I'd be happy to help you with information about NVMe!\n"
 '\n'
 'NVMe (Non-Volatile Memory Express) is a protocol for accessing non-volatile '
 'memory devices, such as solid-state drives (SSDs), using a PCIe interface. '
 'It was developed by a consortium of companies including Intel, Micron '
 'Technology, Samsung, and Seagate, among others.\n'
 '\n'
 'Here are some key features and aspects of the NVMe protocol:\n'
 '\n'
 '1. Fast performance: NVMe is designed to provide much faster performance '
 'than traditional AHCI (Advanced Host Controller Interface) or IDE '
 'interfaces. It uses a specialized controller and a high-speed interface to '
 'transfer data directly between the host and the SSD, bypassing the '
 'traditional disk access layers.\n'
 '2. Low latency: NVMe has lower latency compared to traditional disk '
 'interfaces, which means that it can handle requests for data much more '
 'quickly. This is particularly important for applications that require '
 'frequent

In [9]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

# Prompt
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

result = qa_chain.invoke({"query": "tell me about NVMe command set"})
print(result)

{'query': 'tell me about NVMe command set', 'result': 'The NVMe command set is a standardized set of commands for interacting with NVMe devices, such as solid-state drives (SSDs) and other non-volatile memory devices. The command set is defined by the NVM Express specification, which provides a common language for device manufacturers and operating systems to communicate with NVMe devices.\n\nThe NVMe command set includes several categories of commands:\n\n1. Admin Commands: These commands are used for managing the NVMe device, such as setting configuration options, monitoring device status, and performing maintenance tasks. Examples of admin commands include "Get LBA Status" and "Sanitize".\n2. I/O Command Set Specific Admin Commands: These commands are specific to a particular I/O command set, such as NVMe over PCIe or NVMe over Fabrics. Examples of these commands include "Format NVM" and "Security Send".\n3. Security Commands: These commands are used for securing the NVMe device, su